# 基于graph copy的分布式训练demo

本demo用于演示在使用基于graph copy的分布式训练时，如何从之前保存的meta文件中恢复图结构，并继续训练。


In [1]:
import os
import pandas as pd
import tensorflow as tf
import sys
sys.path.insert(0, "../../")

from rcalgo.tf.training import Model


In [2]:
config = {
    "learning_rate": 0.0001,

    "num_classes": 2,
    "max_seq_len": 100,
    "embedding_size": 256,
    "num_layers": 2,
    "conv_num_filters": 256,
    # "keep_prob": 0.5,

    "gpu_id": 0,
    "gpu_num": 2,

    "model_output_dir": "./saved_models",
    "model_name": "transformer",

    "clip_type": "clip_value",
    "clip_gradient": 1.0
}

In [4]:
# load data
data_dir = "/home/web_server/antispam/project/datasets/female_vulgar_small/"
train_data = pd.read_csv(data_dir + 'train.csv')

In [5]:
# create graph from single replica checkpoint
model = Model(config=config, name="default", distribute="tf_distribute")
single_graph_ckpt = "./saved_models/single_graph/manually_saved_checkpoint.ckpt"
single_graph_meta_path = single_graph_ckpt + ".meta"
if not os.path.exists(single_graph_meta_path):
    print(f"meta file {single_graph_meta_path} not exitsts, please check!")

2020-11-04 18:44:37,927  model.py (73) (MainThread) : INFO  config: {'learning_rate': 0.0001, 'num_classes': 2, 'max_seq_len': 100, 'embedding_size': 256, 'num_layers': 2, 'conv_num_filters': 256, 'gpu_id': 0, 'gpu_num': 2, 'model_output_dir': './saved_models', 'model_name': 'transformer', 'clip_type': 'clip_value', 'clip_gradient': 1.0}


如果之前没有保存checkpoint，可以先使用`GraphCopyCheckpointDemo.py`保存一个基于graph copy的checkpoint。

In [ ]:
# build singel graph from meta file
model.trainer.restore_meta_graph(single_graph_meta_path)
# graph copy and init
model.trainer.initialize()
# load variables from singel graph checkpoint and load to multi-replica graph
model.trainer.load_single_graph_checkpoint(single_graph_ckpt)

2020-11-04 18:44:40,123  single_worker_graph_copy_trainer.py (92) (MainThread) : INFO  Initializing trainer ...
2020-11-04 18:44:40,123  single_worker_graph_copy_trainer.py (105) (MainThread) : INFO  gpu_num: 2


In [ ]:
# train
model.train(
    [train_data['text'], train_data['label']],
    batch_size=1024,
    test_size=0.5,
    epochs=1,
    hooks=[])